##### Copyright 2019 The TensorFlow Authors.

In [91]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text generation with an RNN

We borrowed this code from the following Google TF notebook: https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/text_generation.ipynb

This was a tutorial to demonstrate how to generate text using a character-based RNN.  The example text was Shakespeare.

We modified the notebook for our data and specific needs.

## Setup

### Import TensorFlow and other libraries

In [36]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

In [37]:
print(tf.__version__)

1.13.1


### Download the Shakespeare dataset

Change the following line to run this code on your own data.

In [38]:
path_to_file = './data/relabeled_trajectories_1_workweek.txt'

### Read the data

First, look in the text:

In [39]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
text_len = len(text)
print ('Length of text: {} characters'.format(text_len))
# This is too long for this machine to handle.  Take just a portion of it.
# text = text[:text_len]  # // 2]
print ('using %s characters from text', len(text))

Length of text: 8316581 characters
using %s characters from text 8316581


In [40]:
# Take a look at the first 250 characters in text
print(text[:681])

1 2 1 1 1 1 1 1 1 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 3 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 1 1 1 4 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 1 1 1 1 1 1 0 0 0 0
5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 0 0 0 5 5 5 5 5 5 5 5 5 5 5 5 5 5 0 0 0 0 6 6 6 0 0 0 0 5 5 5 5 5 5 5 5 5 5 5 5 5 5 0 5 0 7 7 8 9 0 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 10 5 5 5 0 11 11 0 11 11 0 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 0 0 0 0 0 0 0 0
12 12 12 12 12 12 12 12 12 12 12 12 13 13 2 14 12 12 12 0 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 13 13 15 15 12 12 12 12 12 3 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 13 13 3 


In [41]:
# The unique characters in the file
vocab = sorted(set(text.split()))
print(vocab)
print ('{} unique characters'.format(len(vocab)))

['0', '1', '10', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '11', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '12', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '13', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '14', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '15', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '16', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '17', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '18', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '19', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '2', '20', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '21', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '22', '220', '221', '222', '223', '224', '225', '226', '227', '228', '22

## Process the text

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [42]:
# Creating a mapping from unique characters to indices
word2idx = {u:i for i, u in enumerate(vocab)}
idx2word = np.array(vocab) # the backwards mapping of char2idx
print(word2idx)

{'0': 0, '1': 1, '10': 2, '100': 3, '101': 4, '102': 5, '103': 6, '104': 7, '105': 8, '106': 9, '107': 10, '108': 11, '109': 12, '11': 13, '110': 14, '111': 15, '112': 16, '113': 17, '114': 18, '115': 19, '116': 20, '117': 21, '118': 22, '119': 23, '12': 24, '120': 25, '121': 26, '122': 27, '123': 28, '124': 29, '125': 30, '126': 31, '127': 32, '128': 33, '129': 34, '13': 35, '130': 36, '131': 37, '132': 38, '133': 39, '134': 40, '135': 41, '136': 42, '137': 43, '138': 44, '139': 45, '14': 46, '140': 47, '141': 48, '142': 49, '143': 50, '144': 51, '145': 52, '146': 53, '147': 54, '148': 55, '149': 56, '15': 57, '150': 58, '151': 59, '152': 60, '153': 61, '154': 62, '155': 63, '156': 64, '157': 65, '158': 66, '159': 67, '16': 68, '160': 69, '161': 70, '162': 71, '163': 72, '164': 73, '165': 74, '166': 75, '167': 76, '168': 77, '169': 78, '17': 79, '170': 80, '171': 81, '172': 82, '173': 83, '174': 84, '175': 85, '176': 86, '177': 87, '178': 88, '179': 89, '18': 90, '180': 91, '181': 92,

In [43]:
def make_text_as_int_list(text):
    results = []
    # first split by new line '\n'
    lines = text.split('\n')
    for i, line in enumerate(lines):
        line_results = []
        area_list = line.split(' ')
        area_indices_list = [word2idx[area] for area in area_list if area is not '']
        for index in area_indices_list:
            line_results.append(index)
#             results.append(char2idx[' '])
        results.append(line_results)  
    return results

vectorized = make_text_as_int_list(text)
vectorized = vectorized[:len(vectorized)-1]
print(vectorized[-1])
print(len(vectorized))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 564, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
23183


Now we have an integer representation for each labeled area.

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

##### NOTE: we don't necessarily add +1


Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [44]:
"""
TODO: we experiment  with what sequence length means!
"""

# The maximum length sentence we want for a single input in characters
seq_length = 122
# examples_per_epoch = len(vectorized)//(seq_length) # (seq_length+1)

# Create training examples / targets
dataset = tf.data.Dataset.from_tensor_slices(vectorized)

for i in dataset.take(5):
  print(idx2word[i.numpy()])

['1' '2' '1' '1' '1' '1' '1' '1' '1' '2' '2' '2' '2' '2' '2' '2' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '2' '2' '2'
 '2' '2' '2' '2' '2' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '2' '2' '2' '2' '2' '2' '2' '3' '3' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '2' '2' '2' '2' '2' '2' '2' '1' '1'
 '1' '4' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '2' '2' '2'
 '2' '2' '2' '2' '1' '1' '1' '1' '1' '1' '0' '0' '0' '0']
['5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5'
 '5' '5' '5' '0' '0' '0' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5'
 '5' '5' '0' '0' '0' '0' '6' '6' '6' '0' '0' '0' '0' '5' '5' '5' '5' '5'
 '5' '5' '5' '5' '5' '5' '5' '5' '5' '0' '5' '0' '7' '7' '8' '9' '0' '5'
 '5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5' '10' '5' '5' '5'
 '0' '11' '11' '0' '11' '11' '0' '5' '5' '5' '5' '5' '5' '5' '5' '5' '5'
 '5' '5' '5' '5' '5' '5' '5' '0' '0' '0' '0' '0' '0' '0' '0']
['1

The `batch` method lets us easily convert these individual characters to sequences of the desired size.

For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [45]:
"""
TODO: experiment with using -3/3 for chunking to represent home & work labels w/ space inbetween 

alsothis  might  not  be  right . because later get: `(500, 675, 654) # (batch_size, sequence_length, vocab_size)`
"""

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = dataset.map(split_input_target)

Print the first examples input and target values:

In [46]:
for input_example, target_example in  dataset.take(1):
  print('Input data: ', input_example)
  print('Target data:', target_example)

Input data:  tf.Tensor(
[  1 112   1   1   1   1   1   1   1 112 112 112 112 112 112 112   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1 112 112 112
 112 112 112 112 112   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1 112 112 112 112 112 112 112 223 223   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1 112 112 112 112 112 112 112   1   1
   1 334   1   1   1   1   1   1   1   1   1   1   1   1   1 112 112 112
 112 112 112 112   1   1   1   1   1   1   0   0   0], shape=(121,), dtype=int32)
Target data: tf.Tensor(
[112   1   1   1   1   1   1   1 112 112 112 112 112 112 112   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1 112 112 112 112
 112 112 112 112   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1 112 112 112 112 112 112 112 223 223   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1 112 112 112 112 112 112 112   1   1   1
 334   1   1   1   1   1   1   1   1   1   1   1   1   1 112 112 11

Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for it and trys to predict the index for the next as the next character. At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.

In [47]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:15], target_example[:15])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2word[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2word[target_idx])))

Step    0
  input: 1 ('1')
  expected output: 112 ('2')
Step    1
  input: 112 ('2')
  expected output: 1 ('1')
Step    2
  input: 1 ('1')
  expected output: 1 ('1')
Step    3
  input: 1 ('1')
  expected output: 1 ('1')
Step    4
  input: 1 ('1')
  expected output: 1 ('1')
Step    5
  input: 1 ('1')
  expected output: 1 ('1')
Step    6
  input: 1 ('1')
  expected output: 1 ('1')
Step    7
  input: 1 ('1')
  expected output: 1 ('1')
Step    8
  input: 1 ('1')
  expected output: 112 ('2')
Step    9
  input: 112 ('2')
  expected output: 112 ('2')
Step   10
  input: 112 ('2')
  expected output: 112 ('2')
Step   11
  input: 112 ('2')
  expected output: 112 ('2')
Step   12
  input: 112 ('2')
  expected output: 112 ('2')
Step   13
  input: 112 ('2')
  expected output: 112 ('2')
Step   14
  input: 112 ('2')
  expected output: 112 ('2')


### Create training batches

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [48]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 121), (64, 121)), types: (tf.int32, tf.int32)>

## Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [49]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
print(vocab_size)

# The embedding dimension  -- TODO: experiment with this size
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

652


In [50]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [51]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](images/text_generation_training.png)

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [52]:
for input_example_batch, target_example_batch in dataset.take(1):
  print(len(input_example_batch))
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

64
(64, 121, 652) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [53]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           166912    
_________________________________________________________________
gru_2 (GRU)                  (64, None, 1024)          3935232   
_________________________________________________________________
dense_2 (Dense)              (64, None, 652)           668300    
Total params: 4,770,444
Trainable params: 4,770,444
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [54]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [55]:
sampled_indices

array([531, 482, 158, 564,   1, 623, 342,  80, 524, 392, 636, 380, 233,
       280,  53, 308, 610, 164, 491, 388, 523,  42, 409,   1, 358, 268,
       579,   9, 171, 557, 404,  10, 220, 203, 249, 284, 135, 518, 549,
        77, 144,   9, 454, 405,  14, 498, 580, 600, 436, 328, 138, 619,
        87,  70, 265, 183, 189, 281, 536,  44, 146,  71, 562, 496, 641,
       345, 127,   5, 327, 332,   0, 151,  22, 491, 338, 291, 636, 568,
       617,   7, 417, 463,  72, 261, 648,  97, 563, 603, 126, 157, 620,
       347, 456, 100, 212, 477,  48, 271, 477, 530, 334, 454, 330, 140,
       333, 426,  68, 252, 139, 112, 187, 542, 558,   4, 239, 344, 314,
       106, 400,  19,  84])

In [56]:
len(sampled_indices)

121

Decode these to see the text predicted by this untrained model:

In [57]:
print("Input: \n", repr(idx2word[input_example_batch[0]]))
print()
print("Next Char Predictions: \n", repr(idx2word[sampled_indices ]))

Input: 
 array(['548', '548', '548', '548', '548', '548', '548', '548', '548',
       '548', '75', '548', '75', '75', '75', '75', '75', '75', '548',
       '548', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '194', '194', '194', '548', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '548', '548', '548', '548',
       '548', '548', '548', '75', '75', '75', '0', '548', '548', '548',
       '548', '316', '316', '548', '548', '548', '548', '548', '548',
       '548', '548', '548', '548', '548', '548', '548', '548', '548',
       '548', '194', '75', '75', '194', '548', '548', '479', '479', '548',
       '548', '548', '548', '548', '548', '548', '548', '548', '548',
       '548', '548', '548', '548', '548', '548', '548', '548', '548',
       '501', '501', '548', '548', '548', '548', '0', '0', '0', '0'],
      dtype='<U3')

Next Char Predictions: 
 array(['577', '532', '240', '606', '1', '73', '406', '170', '570', '451',
       '85', '44

## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the `from_logits` flag.


In [58]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 121, 652)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       6.4817295


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [59]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [60]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints_no_spaces'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [61]:
EPOCHS=100

In [62]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback], steps_per_epoch=3)

Epoch 1/100
2/3 [===================>..........] - ETA: 7s - loss: 6.4309 WARNING:tensorflow:This model was compiled with a Keras optimizer (<tensorflow.python.keras.optimizers.Adam object at 0xb3d7c1ba8>) but is being saved in TensorFlow format with `save_weights`. The model's weights will be saved, but unlike with TensorFlow optimizers in the TensorFlow format the optimizer's state will not be saved.

Consider using a TensorFlow optimizer from `tf.train`.
3/3 [==============================] - 22s 7s/step - loss: 6.3435
Epoch 2/100
2/3 [===================>..........] - ETA: 6s - loss: 7.8971 WARNING:tensorflow:This model was compiled with a Keras optimizer (<tensorflow.python.keras.optimizers.Adam object at 0xb3d7c1ba8>) but is being saved in TensorFlow format with `save_weights`. The model's weights will be saved, but unlike with TensorFlow optimizers in the TensorFlow format the optimizer's state will not be saved.

Consider using a TensorFlow optimizer from `tf.train`.
3/3 [=====

3/3 [==============================] - 20s 7s/step - loss: 3.8898
Epoch 17/100
2/3 [===================>..........] - ETA: 6s - loss: 3.8708 WARNING:tensorflow:This model was compiled with a Keras optimizer (<tensorflow.python.keras.optimizers.Adam object at 0xb3d7c1ba8>) but is being saved in TensorFlow format with `save_weights`. The model's weights will be saved, but unlike with TensorFlow optimizers in the TensorFlow format the optimizer's state will not be saved.

Consider using a TensorFlow optimizer from `tf.train`.
3/3 [==============================] - 20s 7s/step - loss: 3.8589
Epoch 18/100
2/3 [===================>..........] - ETA: 6s - loss: 4.0308 WARNING:tensorflow:This model was compiled with a Keras optimizer (<tensorflow.python.keras.optimizers.Adam object at 0xb3d7c1ba8>) but is being saved in TensorFlow format with `save_weights`. The model's weights will be saved, but unlike with TensorFlow optimizers in the TensorFlow format the optimizer's state will not be saved

3/3 [==============================] - 21s 7s/step - loss: 3.9601
Epoch 33/100
2/3 [===================>..........] - ETA: 6s - loss: 3.7839 WARNING:tensorflow:This model was compiled with a Keras optimizer (<tensorflow.python.keras.optimizers.Adam object at 0xb3d7c1ba8>) but is being saved in TensorFlow format with `save_weights`. The model's weights will be saved, but unlike with TensorFlow optimizers in the TensorFlow format the optimizer's state will not be saved.

Consider using a TensorFlow optimizer from `tf.train`.
3/3 [==============================] - 20s 7s/step - loss: 3.7647
Epoch 34/100
2/3 [===================>..........] - ETA: 6s - loss: 3.7677 WARNING:tensorflow:This model was compiled with a Keras optimizer (<tensorflow.python.keras.optimizers.Adam object at 0xb3d7c1ba8>) but is being saved in TensorFlow format with `save_weights`. The model's weights will be saved, but unlike with TensorFlow optimizers in the TensorFlow format the optimizer's state will not be saved

3/3 [==============================] - 20s 7s/step - loss: 3.1902
Epoch 49/100
2/3 [===================>..........] - ETA: 6s - loss: 2.9958 WARNING:tensorflow:This model was compiled with a Keras optimizer (<tensorflow.python.keras.optimizers.Adam object at 0xb3d7c1ba8>) but is being saved in TensorFlow format with `save_weights`. The model's weights will be saved, but unlike with TensorFlow optimizers in the TensorFlow format the optimizer's state will not be saved.

Consider using a TensorFlow optimizer from `tf.train`.
3/3 [==============================] - 20s 7s/step - loss: 2.9990
Epoch 50/100
2/3 [===================>..........] - ETA: 6s - loss: 3.0755 WARNING:tensorflow:This model was compiled with a Keras optimizer (<tensorflow.python.keras.optimizers.Adam object at 0xb3d7c1ba8>) but is being saved in TensorFlow format with `save_weights`. The model's weights will be saved, but unlike with TensorFlow optimizers in the TensorFlow format the optimizer's state will not be saved

3/3 [==============================] - 20s 7s/step - loss: 1.9193
Epoch 65/100
2/3 [===================>..........] - ETA: 6s - loss: 1.9018 WARNING:tensorflow:This model was compiled with a Keras optimizer (<tensorflow.python.keras.optimizers.Adam object at 0xb3d7c1ba8>) but is being saved in TensorFlow format with `save_weights`. The model's weights will be saved, but unlike with TensorFlow optimizers in the TensorFlow format the optimizer's state will not be saved.

Consider using a TensorFlow optimizer from `tf.train`.
3/3 [==============================] - 20s 7s/step - loss: 1.8353
Epoch 66/100
2/3 [===================>..........] - ETA: 6s - loss: 1.6820 WARNING:tensorflow:This model was compiled with a Keras optimizer (<tensorflow.python.keras.optimizers.Adam object at 0xb3d7c1ba8>) but is being saved in TensorFlow format with `save_weights`. The model's weights will be saved, but unlike with TensorFlow optimizers in the TensorFlow format the optimizer's state will not be saved

3/3 [==============================] - 20s 7s/step - loss: 1.3490
Epoch 81/100
2/3 [===================>..........] - ETA: 6s - loss: 1.3264 WARNING:tensorflow:This model was compiled with a Keras optimizer (<tensorflow.python.keras.optimizers.Adam object at 0xb3d7c1ba8>) but is being saved in TensorFlow format with `save_weights`. The model's weights will be saved, but unlike with TensorFlow optimizers in the TensorFlow format the optimizer's state will not be saved.

Consider using a TensorFlow optimizer from `tf.train`.
3/3 [==============================] - 20s 7s/step - loss: 1.3270
Epoch 82/100
2/3 [===================>..........] - ETA: 6s - loss: 1.2852 WARNING:tensorflow:This model was compiled with a Keras optimizer (<tensorflow.python.keras.optimizers.Adam object at 0xb3d7c1ba8>) but is being saved in TensorFlow format with `save_weights`. The model's weights will be saved, but unlike with TensorFlow optimizers in the TensorFlow format the optimizer's state will not be saved

3/3 [==============================] - 20s 7s/step - loss: 1.0653
Epoch 97/100
2/3 [===================>..........] - ETA: 6s - loss: 1.0392 WARNING:tensorflow:This model was compiled with a Keras optimizer (<tensorflow.python.keras.optimizers.Adam object at 0xb3d7c1ba8>) but is being saved in TensorFlow format with `save_weights`. The model's weights will be saved, but unlike with TensorFlow optimizers in the TensorFlow format the optimizer's state will not be saved.

Consider using a TensorFlow optimizer from `tf.train`.
3/3 [==============================] - 20s 7s/step - loss: 1.0335
Epoch 98/100
2/3 [===================>..........] - ETA: 6s - loss: 1.0786 WARNING:tensorflow:This model was compiled with a Keras optimizer (<tensorflow.python.keras.optimizers.Adam object at 0xb3d7c1ba8>) but is being saved in TensorFlow format with `save_weights`. The model's weights will be saved, but unlike with TensorFlow optimizers in the TensorFlow format the optimizer's state will not be saved

## Generate text

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [63]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints_no_spaces/ckpt_100'

In [64]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [65]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            166912    
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           3935232   
_________________________________________________________________
dense_3 (Dense)              (1, None, 652)            668300    
Total params: 4,770,444
Trainable params: 4,770,444
Non-trainable params: 0
_________________________________________________________________


### The prediction loop

The following code block generates the text:

* It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

* Get the prediction distribution of the next character using the start string and the RNN state.

* Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

* The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one word. After predicting the next word, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted words.


![To generate text the model's output is fed back to the input](images/text_generation_sampling.png)

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [66]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)


  # Number of characters to generate
  num_generate = seq_length

  # Converting our start string to numbers (vectorizing)
  tokens = start_string.split(" "
  print("tokens:", tokens)
  input_eval = [word2idx[s] for s in tokens]
  print('input_eval:')
  print(input_eval)
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2word[predicted_id])

  return (' '.join(tokens + text_generated)) #join with a space

In [67]:
print(generate_text(model, start_string=u"1 2 1 1"))

tokens: ['1', '2', '1', '1']
input_eval:
[1, 112, 1, 1]
1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 495 206 559 405 619 619 214 69 69 619 65 0 113 0 0 0 0 0 358 14 133 0 0 0 0 0 0 0 0 0 170 500 407 468 468 87 0 0 0 0 0 0 0 2 218 0 0 0 0 0 0 0 0 0 0 378 12 113 113 113 113 113 113 0 0 0 0 217 515 236 0 0 0 0 0 0 0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


In [68]:
print(generate_text(model, start_string=u"424 0 424 424 424 424 424 424"))

tokens: ['424', '0', '424', '424', '424', '424', '424', '424']
input_eval:
[362, 0, 362, 362, 362, 362, 362, 362]
424 0 424 424 424 424 424 424 424 424 424 424 424 424 424 424 63 63 63 427 0 90 0 0 0 0 0 333 333 333 333 333 333 193 0 0 468 0 0 0 68 299 299 299 299 299 299 299 0 0 444 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 11 0 0 0 0 0 0 0 0 0 0 0 0 147 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


In [77]:
# Get the counts of each tract in our dataset to see if model is predicting the highest occuring ones more.

text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
lines = text.split('\n')

all_tokens = []
for line in lines:
    tokens = line.split(' ')
    all_tokens += tokens
print(all_tokens[:100])
all_tokens = all_tokens[:len(all_tokens)-1] # last element is empty string ""
print(len(all_tokens))

counts = []
for word in idx2word:
    count = all_tokens.count(word)
    counts.append((word, count))
    
sorted_counts = sorted(counts, key=lambda tup: tup[1])
print(sorted_counts)  

['1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '2', '2', '2', '2', '2', '0', '0', '1', '1', '3', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '4', '1', '1', '5', '5', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '2', '2', '2', '2', '2', '1', '1', '1', '6', '6', '6', '6', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '2', '2', '2', '2', '2', '1', '1', '1', '1', '1', '7', '1', '1', '1', '1', '1', '1']
7854444
[('650', 1), ('651', 32), ('295', 35), ('649', 76), ('395', 92), ('606', 119), ('390', 377), ('578', 554), ('619', 574), ('627', 607), ('103', 686), ('514', 734), ('646', 864), ('642', 898), ('623', 911), ('608', 953), ('226', 965), ('143', 981), ('648', 1013), ('392', 1129), ('200', 1196), ('107', 1252), ('452', 1274), ('501', 1475), ('631', 1482), ('557', 1510), ('234', 1548), ('595', 1579), ('190', 1630), ('562', 1665), ('299', 1679), ('628', 1680), ('485', 1713), ('617', 1714), ('22

The easiest thing you can do to improve the results it to train it for longer (try `EPOCHS=30`).

You can also experiment with a different start string, or try adding another RNN layer to improve the model's accuracy, or adjusting the temperature parameter to generate more or less random predictions.

## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.

So now that you've seen how to run the model manually let's unpack the training loop, and implement it ourselves. This gives a starting point if, for example, to implement _curriculum learning_ to help stabilize the model's open-loop output.

We will use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The procedure works as follows:

* First, initialize the RNN state. We do this by calling the `tf.keras.Model.reset_states` method.

* Next, iterate over the dataset (batch by batch) and calculate the *predictions* associated with each.

* Open a `tf.GradientTape`, and calculate the predictions and loss in that context.

* Calculate the gradients of the loss with respect to the model variables using the `tf.GradientTape.grads` method.

* Finally, take a step downwards by using the optimizer's `tf.train.Optimizer.apply_gradients` method.



In [ ]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [ ]:
optimizer = tf.keras.optimizers.Adam()

In [ ]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [ ]:
# Training step
EPOCHS = 30

for epoch in range(EPOCHS):
  start = time.time()

  # initializing the hidden state at the start of every epoch
  # initally hidden is None
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))